In [2]:
#imports
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

from PathFinder import find_path

In [3]:
#read data
PATH = find_path()

#select files
files = ['application_train.csv','application_test.csv',]
"""         
          'bureau.csv', 
         'bureau_balance.csv', 'credit_card_balance.csv',
         'installments_payments.csv', 'POS_CASH_balance.csv',
         'previous_application.csv']
"""
data_files=[]
data=[]
print("files used: {}".format(files))

#actually read files
for file in files:
    stuff = pd.read_csv(PATH+file,header=0)
    #print(stuff)
    data_files.append(file)
    data.append(stuff)

path: data/
all files in directory: ['application_test.csv', 'application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv', 'sample_submission.csv']
files used: ['application_train.csv', 'application_test.csv']


In [3]:
#select and convert columns
original_train = data[0] #save the original because we'll need some stuff later
original_test = data[1]
app_train = data[0].copy()
app_test = data[1].copy()

#delete ID cols
del app_train['SK_ID_CURR']
del app_test['SK_ID_CURR']

int64cols = app_train.select_dtypes('int64').columns.values.tolist()
objectcols = app_train.select_dtypes('object').columns.values.tolist()
float64cols = app_train.select_dtypes('float64').columns.values.tolist()
#int64cols.remove("SK_ID_CURR")
#print('int cols: {}\n'.format(int64cols))
#print('dbl cols: {}\n'.format(float64cols))
#print('obj cols: {}\n'.format(objectcols))
#print('obj cols in detail:')
#for col in objectcols:
#    print('{}: {}'.format(col, data[0][col].unique()))
#print('\n')

#label encoding for unique <= 2
le = LabelEncoder()
le_count = 0
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            #app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

#one hot encoding
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)
#print('new cols: {}'.format(app_train.columns.values.tolist())) #holy crap thats a lot of bs columns

app_train_y = app_train['TARGET']
new_cols_x = app_train.columns.values.tolist()
new_cols_x.remove('TARGET')
app_train_x = app_train[new_cols_x]

print('Training Features shape: ', app_train_x.shape) #241 cols
print('Testing Features shape: ', app_test.shape) #241 cols

3 columns were label encoded.
Training Features shape:  (307511, 241)
Testing Features shape:  (48744, 241)


In [4]:
# ignore this cell, I thought I was clever and then I read the example in the cell below

# returns an array of values in set1 not in set2
def set_diff(set1, set2):
    arr = []
    for item in set1:
        if item not in set2:
            arr.append(item)
    return arr

#even though columns.length is the same, values arent
train_cols = app_train.columns.values.tolist()
test_cols = app_test.columns.values.tolist()
print('train-test: {}'.format(set_diff(train_cols, test_cols)))
print('test-train: {}'.format(set_diff(test_cols, train_cols)))

train-test: ['TARGET', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CODE_GENDER_XNA', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_FAMILY_STATUS_Unknown']
test-train: ['NAME_CONTRACT_TYPE_Cash loans', 'NAME_CONTRACT_TYPE_Revolving loans', 'FLAG_OWN_CAR_N', 'FLAG_OWN_CAR_Y', 'FLAG_OWN_REALTY_N', 'FLAG_OWN_REALTY_Y']


In [5]:
# Align the training and testing data, keep only columns present in both dataframes
app_train_x, app_test = app_train_x.align(app_test, join = 'inner', axis = 1)

print('Training Features shape: ', app_train_x.shape) #2435 cols
print('Testing Features shape: ', app_test.shape) #235 cols

Training Features shape:  (307511, 235)
Testing Features shape:  (48744, 235)


In [6]:
#impute missing values
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')

app_train_x = imputer.fit_transform(app_train_x)
app_test = imputer.transform(app_test)

In [7]:
#scale values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(app_train_x)
app_train_x = scaler.transform(app_train_x)
app_test = scaler.transform(app_test)

In [8]:
#apply logistic regression
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(app_train_x, app_train_y)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [9]:
# Make predictions
# Make sure to select the second column only
log_reg_pred = log_reg.predict_proba(app_test)[:, 1]

In [10]:
# Submission dataframe
submit = original_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,SK_ID_CURR,TARGET
0,100001,0.063382
1,100005,0.121217
2,100013,0.086837
3,100028,0.057044
4,100038,0.126813


In [11]:
# Save the submission to a csv file
overwrite = False
if overwrite:
    submit.to_csv('results/log_reg_baseline.csv', index = False)

Submitted to kaggle
score: 0.67825

In [12]:
#some BS test. this should be worse than the first, but why is it the same?
submit['TARGET'] = submit['TARGET'] ** 2
overwrite = False
if overwrite:
    submit.to_csv('results/log_reg_baseline_2.csv', index = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
#neural net train
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(300,250,200,150,100,50,40,30,20,10), #possibly too deep lol
                   max_iter=1000)
mlp.fit(app_train_x, app_train_y)

In [ ]:
# Make predictions
mlp_pred = mlp.predict(app_test)
mlp_pred = mlp_pred.clip(0, 1)

In [ ]:
# Submission dataframe
submit = original_test[['SK_ID_CURR']]
submit['TARGET'] = mlp_pred

submit.head()

In [ ]:
# Save the submission to a csv file
overwrite = False
if overwrite:
    submit.to_csv('results/mlp_baseline.csv', index = False)

Submitted to kaggle
score: 0.73529